# Do Data Show Construction Labor Market Shortages?

Construction company executives are increasingly complaining that they are not able to find qualified workers and need government help. While a tight labor market can result in a situation where not enough workers are available, the overall labor force participation rate and employment to population ratio both suggest that there are still unemployed people willing to work. Looking at the data, this is particularly true in the construction industry. With lots of potential workers still on the sidelines, construction employers can easily grow their workforce to meet demand. The obvious way to do this is by offering wages that pull in skilled workers and/or providing training to get new workers up to speed. It seems, however, that executives are trying very different techniques--getting everyone else to pay to train new construction workers and importing low-wage labor from other countries. 

#### Points: 

1) Unemployed per job opening nearly 3x the total for all jobs.<br>
2) Average weekly hours are basically flat and currently lower than 2015 level.<br>
3) Real wage growth is below total private.<br>
4) Construction is seasonal (average hours worked by month) and very subject to swings in the economy (construction layoffs vs total).<br>
5) Construction will be slowed by Fed rate hikes (higher borrowing costs for builders).

#### Policies that increase the supply of low-wage workers:

1) Training programs that put too much emphasis on the skills that executives demand rather than the skills which most benefit workers. <br>
2) Visas to bring in low-wage workers from overseas in cases where there are plenty of domestic workers with the appropriate skills. 

#### Technical preliminaries

Python 2.7

In [33]:
import requests
import json
import pandas as pd
import config # file which contains API keys
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.grid_objs import Column, Grid

plotly.tools.set_credentials_file(username='bdew', api_key=config.plotly_key)

#### Request data from BLS

In [2]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)
headers = {'Content-type': 'application/json'}   # Request json data
sd = {
    'TOT_AHE': 'CES0500000008',  # Average hourly earnings
    'CON_AHE': 'CES2000000008',
    'CON_JOU': 'JTU23000000JOL', # Job openings (level) NSA
    'TOT_JOU': 'JTU00000000JOL',
    'CON_UN': 'LNU03032231',     # Unemployed persons NSA
    'TOT_UN': 'LNU03000000',
    'CON_HRS': 'CES2000000007',  # Weekly hours
    'CPI-U': 'CUSR0000SA0'      # CPI for all urban consumers
}

In [3]:
data = json.dumps({"seriesid":sd.values(), "startyear":'2014', "endyear":'2017'})
p = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()

#### Build pandas dataframe from results

In [4]:
# Empty dictionary. Each entry will be one series
d = {}

# Loop through the series and convert to datetime indexed float values
for series in p['Results']['series']:
    s = series['seriesID']  # Short name, 's', for series ID
    d[s] = pd.DataFrame(series['data'])
    d[s]['date'] = pd.to_datetime(d[s]['period'] + ' ' + d[s]['year'])
    d[s] = d[s].set_index('date')['value'].astype(float)

# Combine the individual dictionary entries into one dataframe
df = pd.concat(d, axis=1)

# Show last five rows
df.tail()

,CES0500000008,CES2000000007,CES2000000008,CUSR0000SA0,JTU00000000JOL,JTU23000000JOL,LNU03000000,LNU03032231
date,,,,,,,,
2017-02-01,21.86,39.8,26.29,244.456,5500.0,181.0,7887.0,781.0
2017-03-01,21.90,39.3,26.40,243.752,5723.0,161.0,7284.0,764.0
2017-04-01,21.96,39.9,26.37,244.158,6312.0,200.0,6555.0,585.0
2017-05-01,21.99,39.9,26.51,243.846,5665.0,154.0,6572.0,502.0
2017-06-01,22.03,39.7,26.64,243.790,NaN,NaN,7250.0,430.0


#### Basic calculations for real wages and unemployed per job opening

In [5]:
# Real wages for construction (CON) and total private
df['CON_RW_12CH'] = df[sd['CON_AHE']].multiply(
    df[sd['CPI-U']][0]).divide(
    df[sd['CPI-U']]).pct_change(12).multiply(100)
df['TOT_RW_12CH'] = df[sd['TOT_AHE']].multiply(
    df[sd['CPI-U']][0]).divide(
    df[sd['CPI-U']]).pct_change(12).multiply(100)
# Unemployed per job opening
df['CON_UNOP'] = df[sd['CON_UN']].divide(df[sd['CON_JOU']])
df['TOT_UNOP'] = df[sd['TOT_UN']].divide(df[sd['TOT_JOU']])

#### Save data to a csv file

In [6]:
df.to_csv('construction_data.csv')

## Review the results

#### Unemployed per job opening

Down overall, up for construction over the past year

In [7]:
# One year ago
df[['CON_UNOP', 'TOT_UNOP']].dropna().iloc[-13]

CON_UNOP    2.465241
TOT_UNOP    1.289036
Name: 2016-05-01 00:00:00, dtype: float64

In [8]:
# Latest Available
df[['CON_UNOP', 'TOT_UNOP']].dropna().iloc[-1]

CON_UNOP    3.259740
TOT_UNOP    1.160106
Name: 2017-05-01 00:00:00, dtype: float64

In [9]:
# Plot will go here

#### Hours worked

Flat over the past three years

In [10]:
df[sd['CON_HRS']].iloc[-37:]#.plot()

date
2014-06-01    39.6
2014-07-01    39.9
2014-08-01    39.8
2014-09-01    39.7
2014-10-01    39.3
2014-11-01    39.7
2014-12-01    39.8
2015-01-01    39.7
2015-02-01    39.7
2015-03-01    39.4
2015-04-01    39.3
2015-05-01    39.4
2015-06-01    39.8
2015-07-01    39.5
2015-08-01    39.8
2015-09-01    39.6
2015-10-01    40.0
2015-11-01    39.7
2015-12-01    40.2
2016-01-01    39.8
2016-02-01    39.6
2016-03-01    39.4
2016-04-01    39.5
2016-05-01    39.6
2016-06-01    39.7
2016-07-01    39.7
2016-08-01    39.4
2016-09-01    39.6
2016-10-01    39.6
2016-11-01    39.8
2016-12-01    39.2
2017-01-01    39.3
2017-02-01    39.8
2017-03-01    39.3
2017-04-01    39.9
2017-05-01    39.9
2017-06-01    39.7
Freq: MS, Name: CES2000000007, dtype: float64

In [35]:
awe_series = pd.DataFrame(df[sd['CON_HRS']].iloc[-37:])
awe_series['new'] = ''
awe_series['new'].iloc[0] = awe_series['CES2000000007'][0]
awe_series['new'].iloc[-1] = awe_series['CES2000000007'][-1]

In [39]:
# Average Weekly Hours Plot

trace1 = {
  "x": awe_series.index, 
  "y": awe_series['CES2000000007'], 
  "mode": "lines", 
  "name": "Average Weekly Hours, Construction", 
  "text": ["65+: 1993: 7.9%", "65+: 1994: 8.8%", "65+: 1995: 8.5%", "65+: 1996: 8.2%", "65+: 1997: 8.3%", "65+: 1998: 8.4%", "65+: 1999: 8.7%", "65+: 2000: 9.1%", "65+: 2001: 9.3%", "65+: 2002: 9.5%", "65+: 2003: 10.2%", "65+: 2004: 10.7%", "65+: 2005: 11.1%", "65+: 2006: 11.3%", "65+: 2007: 12.3%", "65+: 2008: 12.7%", "65+: 2009: 12.8%", "65+: 2010: 12.9%", "65+: 2011: 13.1%", "65+: 2012: 13.5%"], 
  "type": "scatter"
}
trace2 = {
  "x": awe_series.index,
  "y": awe_series['new'], 
  "hoverinfo": "none", 
  "marker": {"size": 10}, 
  "mode": "markers+text", 
  "name": "", 
  "text": awe_series['new'],  
  "textfont": {"size": 16}, 
  "textposition": "top center", 
  "type": "scatter", 
}
data = Data([trace1, trace2])
layout = {
  "annotations": [
    {
      "x": -0.0942857142857, 
      "y": -0.253550543024, 
      "align": "left", 
      "font": {
        "family": "Open Sans, sans-serif", 
        "size": 10
      }, 
      "showarrow": False, 
      "text": "http://cepr.net<br>Source: Bureau of Labor Statistics, labor force statistics from the current population survey.  ", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.0895238095238, 
      "y": 1.2576984127, 
      "align": "left", 
      "font": {
        "family": "Open Sans", 
        "size": 15
      }, 
      "showarrow": False, 
      "text": "<b>Construction worker hours are flat over<br>the past three years</b>", 
      "xref": "paper", 
      "yref": "paper"
    }
  ], 
  "autosize": False, 
  "height": 300, 
  "hovermode": "closest", 
  "legend": {
    "x": 0.616334319709, 
    "y": 0.306177606176, 
    "font": {"size": 12}, 
    "orientation": "v", 
    "traceorder": "normal"
  }, 
  "margin": {
    "r": 10, 
    "t": 50, 
    "b": 70, 
    "l": 40
  }, 
  "paper_bgcolor": "rgb(255, 255, 255)", 
  "plot_bgcolor": "rgb(255, 255, 255)", 
  "showlegend": False, 
  "title": "", 
  "titlefont": {
    "size": 16
  }, 
  "width": 400, 
  "xaxis": {
    "anchor": "y", 
    "autorange": False, 
    "fixedrange": True, 
    "nticks": 10, 
    "range": ["2014-03-01", "2017-09-01"], 
    "showgrid": False, 
    "side": "bottom", 
    "tickfont": {
      "color": "rgb(127, 127, 127)", 
      "family": "Open Sans"
    }, 
    "ticks": "inside", 
    "ticksuffix": "", 
    "title": "", 
    "type": "date", 
    "zeroline": False
  }, 
  "yaxis": {
    "fixedrange": True, 
    "range": [30, 50], 
    "showgrid": False, 
    "showticklabels": False, 
    "tick0": 0, 
    "tickmode": "linear", 
    "ticks": "", 
    "ticksuffix": "%", 
    "title": "", 
    "type": "linear"
  }
}
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='Construction_Labor_Market_fig2')

#### Real wage growth

About the same in construction as in all sectors -- very low!

In [12]:
df[['TOT_RW_12CH', 'CON_RW_12CH']].dropna().iloc[-1:]

,TOT_RW_12CH,CON_RW_12CH
date,,
2017-06-01,0.665306,0.608771


In [13]:
# Plot will go here